[ Stefano Pigozzi | Tema Data Analytics | Big Data Analytics | A.A. 2022/2023 | Unimore ]

# Attività online correlata all'avanzare della pandemia COVID

> ### Data processing e exploratory data analytics su dataset provenienti da più sorgenti
> 
> L’attività da svolgere consiste nel:
> 1. Scegliere due o più dataset provenienti da due o più sorgenti.  
>     * Il dataset finale deve essere costituito almeno da due file.
> 2. Usando [pandas](https://pandas.pydata.org/) implementare le operazioni di data processing necessarie (principalmente join e selezioni) per mettere in collegamento i dataset e per preparare i dati al passo successivo
> 3. Usando pacchetti Python quali [pandas](https://pandas.pydata.org/), [scipy](https://scipy.org/), [matplotlib](https://matplotlib.org/) e [seaborn](https://seaborn.pydata.org/) implementare attività di data cleaning, exploratory data analysis estraendo dati statistici e di visualizzazione dei risultati attraverso il quale sia possibile "raccontare qualcosa sui dati" (storytelling), eventualmente partendo da dei quesiti di ricerca. L'uso
dei pacchetti non deve necessariamente essere limitato alle istruzioni viste a lezione. Le documentazioni dei pacchetti stessi e i volumi messi a disposizione su Dolly fornisco spunti d’uso interessanti!
> 4. Produrre un notebook Jupyter (https://jupyter.org/) che contenga:
>     * una introduzione all’argomento scelto, alle sorgenti dati e agli obiettivi del progetto specificando
eventualmente i quesiti di ricerca
>     * una sezione per ogni fase del progetto di data analytics

## Abstract

<!-- TODO -->

## Installazione requisiti

Per funzionare, questo progetto necessita di alcuni package Python, scaricabili da [PyPI](https://pypi.org/).

In particolare:

* [numpy](https://pypi.org/project/numpy/1.23.5/)
* [scipy](https://pypi.org/project/scipy/1.9.3/)
* [pandas](https://pypi.org/project/pandas/1.5.2/)
* [matplotlib](https://pypi.org/project/matplotlib/3.6.2/)

In [ ]:
%pip install numpy==1.23.5 scipy==1.9.3 pandas==1.5.2 matplotlib==3.6.2

### Import

Nella seguenti celle, si effettua l'`import` di tutti i package utilizzati, in modo da poterli utilizzare nel presente documento Jupyter.

In [ ]:
import numpy
numpy

In [ ]:
import scipy
scipy

In [ ]:
import pandas
pandas

In [ ]:
import matplotlib
matplotlib

Si importano anche alcuni package della standard library di Python utili allo sviluppo:

In [ ]:
import pathlib
pathlib

In [ ]:
import typing as t
t

In [ ]:
import io
io

In [ ]:
import json
json

In [ ]:
import datetime
datetime

In [ ]:
import pickle
pickle

## Introduzione

<!-- TODO -->

## Fonti dati

### Google Trends

[Google Trends](https://trends.google.com/trends/) è un servizio messo a disposizione da Google che permette di ispezionare il volume di ricerca nel tempo e nello spazio per dati termini.

Selezionato un termine o un argomento di ricerca, è possibile esportare un file CSV contenente il volume relativo di ricerca nel tempo, con !!!intervalli!!! variabili da 1 mese a 1 minuto.

#### Esempio

Si allega un esempio di file CSV [esportato da Google Trends per la ricerca "Google"](https://trends.google.com/trends/explore?date=all&q=Google):

```csv
Categoria: Tutte le categorie

Mese,Google: (Tutto il mondo)
2004-01,6
2004-02,6
2004-03,7
2004-04,8
2004-05,7
2004-06,7
2004-07,7
2004-08,7
2004-09,8
2004-10,8
2004-11,8
2004-12,8
2005-01,8
2005-02,9
[...]
```

#### Importazione dati

Si definisce una funzione, `import_gtrends`, in grado di caricare dati scaricati da Google Trends in un [`pandas.DataFrame`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html).

In [ ]:
def import_gtrends(fd: t.IO[str]) -> pandas.DataFrame:
   """
   Import a Google Trends CSV file into a :class:`pandas.Series`.

   :param fd: The file descriptor of the CSV file.
   :return: The imported :class:`pandas.Series`.
   """
   return pandas.read_csv(fd, sep=",", header=1)

Si fornisce un esempio del funzionamento di questa funzione:

In [ ]:
with open("data/gtrends/google-worldwide.csv") as file:
   google_worldwide = import_gtrends(file)

google_worldwide

### Telegram

[Telegram](https://telegram.org/), per conformità al GDPR, permette di [esportare i contenuti di una o più chat](https://t.me/gdprbot) in cui si è presenti in un file JSON leggibile da calcolatori.

#### Esempio

Si allega un esempio di file JSON esportato da Telegram per una chat creata appositamente per lo scopo:

```json
{
 "name": "Esempio",
 "type": "private_group",
 "id": 660884870,
 "messages": [
  {
   "id": 1531670,
   "type": "service",
   "date": "2022-12-02T14:49:03",
   "date_unixtime": "1669988943",
   "actor": "Steffo",
   "actor_id": "user25167391",
   "action": "create_group",
   "title": "Esempio",
   "members": [
    "Steffo"
   ],
   "text": "",
   "text_entities": []
  },
  {
   "id": 1531671,
   "type": "message",
   "date": "2022-12-02T14:49:11",
   "date_unixtime": "1669988951",
   "from": "Steffo",
   "from_id": "user25167391",
   "text": "Questo è un messaggio di esempio.",
   "text_entities": [
    {
     "type": "plain",
     "text": "Questo è un messaggio di esempio."
    }
   ]
  }
 ]
}
```

#### Importazione dati

Il file esportato da Telegram contiene tutte le informazioni di ogni evento di quella chat, inclusi gli autori e i contenuti dei messaggi: sono **dati sensibili**, e non possono essere inclusi per motivi di privacy.

Visto però che l'unico dato rilevante ai fini di questa ricerca è la **data di invio di ciascun messaggio**, tutte le informazioni possono essere rimosse dal dataset, rendendolo così privo di informazioni sensibili.

Si definisce una funzione, `anonymize_telegram`, in grado di leggere i file JSON esportati da Telegram e di restituire da essi una lista di date in cui sono stati inviati messaggi.

In [ ]:
def anonymize_telegram(fd: t.IO[str]) -> list[datetime.datetime]:
   """
   Read a Telegram GDPR export file, stripping everything but the messages' dates from it, returning a :class:`list` of :class:`datetime.datetime`s.

   :param fd: The input file descriptor.
   :returns:
   """
   data = json.load(fd)
   msgs = data["messages"]

   def msg_to_date(msg: dict[str, t.Any]) -> datetime.datetime:
      """
      Convert a message :class:`dict` to the :class:`datetime.datetime` of its creation.

      :param msg: The message to convert.
      :return: The resulting :class:`datetime.datetime`.
      """
      return datetime.datetime.fromisoformat(msg["date"])

   return list(map(msg_to_date, msgs))

Si fornisce un esempio del funzionamento di questa funzione:

In [ ]:
with open("data/telegram/example.json") as file:
   telegram_example = anonymize_telegram(file)

telegram_example

I dati così anonimizzati possono essere allegati alla relazione: si definisce una funzione `store_telegramanon` in grado di archiviare le liste di date create dalla precedente funzione in un file formato Pickle.

In [ ]:
def store_telegramanon(l: list[datetime.datetime], fd: t.IO[bytes]) -> None:
   """
   Store a :class:`list` generated by :func:`anonymize_telegram` in a binary file descriptor.

   :param l: The :class:`list` to store.
   :param fd: The file descriptor to write to.
   """
   pickle.dump(l, fd)

Si fornisce un esempio del funzionamento di questa funzione:

In [ ]:
with open("data/telegramanon/example.pickle", "wb") as file:
   store_telegramanon(telegram_example, file)

Infine, si definisce una funzione, `import_telegramanon`, in grado di leggere i dati precedentemente archiviati in un [`pandas.DataFrame`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html):

In [ ]:
def import_telegramanon(fd: t.IO[bytes]) -> pandas.DataFrame:
   """
   Import a :class:`list` stored by :func:`store_telegramanon` into a :class:`pandas.DataFrame`.

   :param fd: The file descriptor to read from.
   :return: The imported :class:`pandas.Series`.
   """
   data = pickle.load(fd)
   return pandas.DataFrame(data)

Ancora, si fornisce un esempio del funzionamento di questa funzione:

In [ ]:
with open("data/telegramanon/example.pickle", "rb") as file:
   telegramanon_example = import_telegramanon(file)

telegramanon_example

### Dipartimento della Protezione Civile della Presidenza del Consiglio dei Ministri

Il Dipartimento della Protezione Civile [mette a disposizione un repository Git contenente i dati relativi alla diffusione del COVID-19 in Italia](https://github.com/pcm-dpc/COVID-19).

All'interno del repository Git, i dati sono disponibili in formato CSV.

Il repository è allegato a questo progetto come Git Submodule.

Essendo di dimensioni significative, di default non viene scaricato da Git, ma è possibile effetuarne il download con i seguente comandi:

In [44]:
!git submodule init
!git submodule update

#### Esempio

Si allega un esempio di file CSV presente all'interno del repository:

```csv
data,stato,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note,ingressi_terapia_intensiva,note_test,note_casi
2020-02-24T18:00:00,ITA,101,26,127,94,221,0,221,1,7,,,229,4324,,,,,
```

#### Importazione dati

Ai fini di questa relazione, si è particolarmente interessati ai dati contenuti nel file `data/covid19/dati-json/dpc-covid19-ita-andamento-nazionale.json`, che contiene le statistiche delle infezioni di COVID-19 a livello nazionale.

Si definisce una funzione `import_covid19`, che importa i dati di quel file JSON in un [`pandas.DataFrame`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html):

In [ ]:
def import_covid19(fd: t.IO[str]) -> pandas.DataFrame:
   """
   Import data from the given file descriptor to ``dpc-covid19-ita-andamento-nazionale.json`` into a :class:`pandas.DataFrame`.

   :param fd: The file descriptor.
   :return: The :class:`pandas.DataFrame`.
   """
   return pandas.read_json(fd)

Si effettua immediatamente l'importazione dei dati nella variabile `covid19_frame`, trovandosi essi in un file unico:

In [ ]:
with open("data/covid19/dati-json/dpc-covid19-ita-andamento-nazionale.json") as file:
   covid19_frame = import_covid19(file)

covid19_frame